In [6]:
import numpy as np #Importation des packages et bibliothèques
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
from scipy.stats import f_oneway, kruskal
from sklearn.ensemble import IsolationForest

dataSet = sp.loadmat('breastw.mat') # On importe le dataset 
X = dataSet['X'] # On récupère les caractéristiques des points du data
y = dataSet['y'] # On récupère les labels des points du dataset

Contamination = [0.35] #Création des plages de valeurs des paramètres optimisés
nombre_estim =[30,30,40,50,10,20,30,30,30,40]
MS=[50,20,40,30,10,40,40,100,100,30]
plage_features = [2,6,5,2,9,4,4,4,7,6]
skf= StratifiedKFold(n_splits=10,shuffle=False,random_state=None) # On splits nos données en 10
for i in range(0,9) :  # boucle for pour changer de combinaison, 10 combinaisons 
        test=False
        for train_index, test_index in skf.split(X, y):
                X_train, X_test = X[train_index,:], X[test_index,:]  #Séparation des données en données d'entrainement et de test
                y_train, y_test = y[train_index], y[test_index]
                model = IsolationForest(contamination=Contamination[0], n_estimators=nombre_estim[i], max_samples=MS[i],max_features=plage_features[i]) #Paramétrage du modèle Isolation Forest
                model.fit(X_train)
                y_pred=model.predict(X_test) #Récupération des labels prédit de chaque points
                y_pred[y_pred>0]=0 #le modèle renvoie 1 et -1 comme label, conversion en 0 et 1 pour le calcul du F1 score
                y_pred[y_pred<0]=1
                F1=round(f1_score(y_test,y_pred),3) #Calcul du F1 score
                if test == False :
                        matrice = np.array([[F1]])  #Construction d'une matrice de stockage des F1 scores, réalisé qu'une fois grâce au boolean test
                        test = True
                else :
                        matrice = np.append(matrice,np.array([F1]).reshape(1,1),axis=0) #Ajout des F1 score à la matrice
        if i == 0  :
                df=pd.DataFrame(matrice,columns=['[{},{},{},{}]'.format(round(Contamination[0],3),nombre_estim[i],MS[i],plage_features[i])]) #Création de data frames pour contenir nos résultats
        else :
                df1=pd.DataFrame(matrice,columns=['[{},{},{},{}]'.format(round(Contamination[0],3),nombre_estim[i],MS[i],plage_features[i])])
                df=pd.concat([df, df1], axis = 1)
display(df) #Affichage des F1 scores des diffférentes combinaisons     

,"[0.35,30,50,2]","[0.35,30,20,6]","[0.35,40,40,5]","[0.35,50,30,2]","[0.35,10,10,9]","[0.35,20,40,4]","[0.35,30,40,4]","[0.35,30,100,4]","[0.35,30,100,7]"
0,0.980,0.894,0.917,0.844,0.960,0.826,0.894,0.917,0.917
1,0.939,0.898,0.917,0.917,0.851,0.894,0.875,0.920,0.941
2,0.923,0.941,0.941,0.920,0.875,0.960,0.941,0.941,0.920
3,0.894,0.936,0.894,0.917,0.894,0.870,0.913,0.936,0.936
4,0.939,0.960,0.917,0.917,0.960,0.939,0.939,0.917,0.913
5,0.920,0.920,0.913,0.913,0.875,0.875,0.851,0.917,0.939
6,0.979,0.979,0.913,0.979,0.979,0.936,0.958,0.933,0.957
7,1.000,1.000,0.979,0.979,0.958,0.958,1.000,0.958,0.958
8,0.979,0.979,0.957,0.979,0.979,0.960,0.979,0.957,0.979
9,0.980,0.980,0.980,1.000,0.979,0.960,0.958,0.980,0.980


In [7]:
from scipy.stats import f_oneway, kruskal
from scipy import stats
import scikit_posthocs as sp #Importation des packages et bibliothèques
t,p = kruskal(df['[0.35,30,50,2]'],df['[0.35,30,20,6]'],df['[0.35,40,40,5]'],df['[0.35,50,30,2]'],df['[0.35,10,10,9]'],df['[0.35,20,40,4]'],df['[0.35,30,40,4]'],df['[0.35,30,100,4]'],df['[0.35,30,100,7]'])
print('t statistic: %.3f' % t) #Affichage résultat tests statistiques
print('p value: %.3f' % p)    

sp.posthoc_conover([df['[0.35,30,50,2]'],df['[0.35,30,20,6]'],df['[0.35,40,40,5]'],df['[0.35,50,30,2]'],df['[0.35,10,10,9]'],df['[0.35,20,40,4]'],df['[0.35,30,40,4]'],df['[0.35,30,100,4]'],df['[0.35,30,100,7]']]) #Posthoc

t statistic: 5.462
p value: 0.707


,1,2,3,4,5,6,7,8,9
1,1.000000,0.739248,0.120663,0.267667,0.251821,0.074751,0.179857,0.243311,0.438890
2,0.739248,1.000000,0.220584,0.436415,0.414518,0.145095,0.311289,0.402643,0.658288
3,0.120663,0.220584,1.000000,0.652212,0.679735,0.813445,0.829829,0.695218,0.431491
4,0.267667,0.436415,0.652212,1.000000,0.969742,0.492714,0.813445,0.952948,0.736072
5,0.251821,0.414518,0.679735,0.969742,1.000000,0.516841,0.842992,0.983187,0.707700
6,0.074751,0.145095,0.813445,0.492714,0.516841,1.000000,0.652212,0.530509,0.307316
7,0.179857,0.311289,0.829829,0.813445,0.842992,0.652212,1.000000,0.859507,0.566902
8,0.243311,0.402643,0.695218,0.952948,0.983187,0.530509,0.859507,1.000000,0.692111
9,0.438890,0.658288,0.431491,0.736072,0.707700,0.307316,0.566902,0.692111,1.000000
